# Script Writing

GPT-2 Model Experiment 2<br>
Script writing in Korean
- Data: [짤툰](https://www.youtube.com/c/%EC%A7%A4%ED%88%B01) script data
- Model: [SKT AI KoGPT2](https://github.com/SKT-AI/KoGPT2) fine-tuning

Author: [Seongbum Seo](https://github.com/Seongbuming)

In [1]:
import torch
torch.cuda.empty_cache()

## Background Setup

In [5]:
# Install transformers library
%pip install -q git+https://github.com/huggingface/transformers.git
# Install helper functions
%pip install -q git+https://github.com/gmihaila/ml_things.git
%pip install -q fastai==2.2.5

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Clone base model
!git clone https://github.com/SKT-AI/KoGPT2
%pip install matplotlib==3.1.3

fatal: destination path 'KoGPT2' already exists and is not an empty directory.
     |████████████████████████████████| 13.1 MB 6.8 MB/s eta 0:00:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

ml-things 0.0.1 requires matplotlib>=3.4.0, but you'll have matplotlib 3.1.3 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Model setup

In [12]:
import io
import os
import torch
import transformers
import fastai
import re
from typing import Optional
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast
from fastai.text.all import *

# Number of training epochs
epochs = 10

# Number of batches - depending on the max sequence length and GPU memory
# For 512 sequence length batch of 10 works without cuda memory issues
# For small sequence length can try batch of 32 or higher
batch_size = 8

# Pad or truncate text sequences to a specific length
# If 'None' it will use maximum sequence of word piece tokens allowed by model
max_length = 256

# Look for GPU to use
# Will use 'cpu' by default if no GPU found
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Name of the base model to use
model_name_or_path = 'skt/kogpt2-base-v2'

# Path of data to use for training
data_path = './dataset/jjaltoon_scripts'

## Data

In [11]:
class ScriptDataset(Dataset):
    def __init__(self, path, use_tokenizer):
        # Check if path exists
        if not os.path.isdir(path):
            # Raise error if path is invalid
            raise ValueError('Invalid `path` variable. Needs to be a directory.')
        
        self.scripts = []
        
        # Get all files from path
        files_names = os.listdir(path)
        # Go through each file and read its content
        for file_name in tqdm(files_names, desc=f'script files'):
            file_path = os.path.join(path, file_name)
            
            # Read content
            content = io.open(file_path, mode='r', encoding='utf-8').read()
            # Fix any unicode issues
            content = fix_content(content)
            # Save content
            self.scripts.append(content)
        
        # Number of examples
        self.n_sexamples = len(self.examples)
    
    def __len__(self):
        r'''When used `len` return the number of examples.
        '''
        
        return self.n_examples
    
    def __getitem__(self, item):
        r'''Given an index return an example from the position.
        
        Arguments:
            item(:obj:`int`):
                Index position to pick an example to return.
        
        Returns:
            :obj:`str`: Script of the index position.
        '''
        
        return self.scripts[item]

In [ ]:
class Gpt2ScriptWritingCollator(object):
    r'''Data Collator used for GPT-2 in a script writing task.
    
    It uses a given tokenizer and its encoder to convert any text to numbers that can go straight into a GPT-2 model.
    
    Arguments:
        use_tokenizer(:obj:`transformers.tokenization_?`):
            Transformer type tokenizer used to process raw text into numbers
        max_sequence_len(:obj:`int`, `optional`):
            Value to indicate the maximum desired sequence to truncate or pad text sequences.
            If no value is passed it will used maximum sequence size supported by the tokenizer and model.
    '''
    
    def __init__(self, use_tokenizer, max_sequence_len=None):
        # Tokenizer to be used inside the class
        self.use_tokenizer = use_tokenizer
        # Check max sequence length
        self.max_sequence_len = use_tokenizer.model_max_length if max_sequence_len is None else max_sequence_len
    
    def __call__(self, sequences):
        r'''This function allowes the class object to be used as a function call.
        Since the PyTorch DataLoader needs a collator function, can use this class as a function.
        
        Arguments:
            item(:obj:`list`):
                List of texts.
        
        Returns:
            :obj:`Dict[str, object]`: Dictionary of inputs that feed into the model.
            It holds the statement `model(**Returned Dictionary)`.
        '''
        
        # Get all texts from sequences list
        text = [sequence['text'] for sequence in sequences]
        # Call tokenizer on all texts to convert into tensors of numbers with appropriate padding
        inputs = self.use_tokenizer(text=text, return_tensors='pt', padding=True, truncate=True, max_length=self.max_sequence_len)
        
        return inputs